In [1]:
import tensorflow as tf

In [72]:
with open("read_tf.txt","r") as filer:
    print(filer.read())

I use Tensorflow
You use PyTorch
Both are great


In [12]:
#tf.data is used to build input pipelines
#Former data pipelines made the GPU wait for the CPU to load the data, leading to performance issues.

In [48]:
input_data=tf.data.TextLineDataset("read_tf.txt")


In [49]:
iterate=input_data.make_one_shot_iterator()
next_data=iterate.get_next()
with tf.Session() as sess:
    for i in range(3):
        print(sess.run(next_data))

b'I use Tensorflow'
b'You use PyTorch'
b'Both are great'


In [50]:
#Splitting words by space 
input_data=input_data.map(lambda x:tf.string_split([x]).values)


In [51]:
iterate=input_data.make_one_shot_iterator()
next_data=iterate.get_next()
with tf.Session() as sess:
    for i in range(3):
        print(sess.run(next_data))

[b'I' b'use' b'Tensorflow']
[b'You' b'use' b'PyTorch']
[b'Both' b'are' b'great']


In [52]:
#Shuffling dataset
input_data=input_data.shuffle(buffer_size=3) #buffer size is to load data in batches
iterate=input_data.make_one_shot_iterator()
next_data=iterate.get_next()
with tf.Session() as sess:
    for i in range(3):
        print(sess.run(next_data))


[b'You' b'use' b'PyTorch']
[b'Both' b'are' b'great']
[b'I' b'use' b'Tensorflow']


In [53]:
#create batches
new_data_batch=input_data.batch(batch_size=2)#to load data in batches
fetcher=new_data_batch.prefetch(buffer_size=1)#always have data in the pipeline
iterate=fetcher.make_one_shot_iterator()
next_batch_data=iterate.get_next()
with tf.Session() as sess:
    for i in range(3):
        try:
            print(sess.run(next_batch_data))
        except:continue

[[b'Both' b'are' b'great']
 [b'You' b'use' b'PyTorch']]
[[b'I' b'use' b'Tensorflow']]


In [55]:
#we use initializable iterators so that we can chose to “restart” from the beginning.
#This will become quite handy when we want to perform multiple epochs !

In [56]:
dataset = tf.data.TextLineDataset("read_tf.txt")
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
init_op = iterator.initializer
with tf.Session() as sess:
    # Initialize the iterator
    sess.run(init_op)
    print(sess.run(next_element))
    print(sess.run(next_element))
    # Move the iterator back to the beginning
    sess.run(init_op)
    print(sess.run(next_element))

b'I use Tensorflow'
b'You use PyTorch'
b'I use Tensorflow'


In [73]:
#Building an image data pipeline

In [83]:
#.Create the dataset from slices of the filenames and labels
#.Shuffle the data with a buffer size equal to the length of the dataset. This ensures good shuffling 
#.Parse the images from filename to the pixel values. Use multiple threads to improve the speed of preprocessing
#.(Optional for training) Data augmentation for the images. Use multiple threads to improve the speed of preprocessing
#.Batch the images
#.Prefetch one batch to make sure that a batch is ready to be served at all time

In [89]:
#step 1
import numpy as np
features, labels = (np.random.sample((100,2)), np.random.sample((100,1)))
dataset = tf.data.Dataset.from_tensor_slices((features,labels))

In [106]:
sequence = np.array([[[1]],[[2],[3]],[[3],[4],[5]]])
def generator_seq():
    for el in sequence:
        yield el
dataset_seq = tf.data.Dataset().batch(1).from_generator(generator_seq,
                                           output_types= tf.int64, 
                                           output_shapes=(tf.TensorShape([None, 1])))


In [107]:
iter = dataset_seq.make_initializable_iterator()
el = iter.get_next()
with tf.Session() as sess:
    sess.run(iter.initializer)
    print(sess.run(el))
    print(sess.run(el))
    print(sess.run(el))

[[1]]
[[2]
 [3]]
[[3]
 [4]
 [5]]


In [112]:
#from csv
import pandas as pd
df=pd.read_csv("sample.csv")
df.head()

,Start Date,Start Time,End Date,End Time,Event Title,All Day Event,No End Time,Event Description,Contact,Contact Email,Contact Phone,Location,Category,Mandatory,Registration,Maximum,Last Date To Register
0,9/5/2011,3:00:00 PM,9/5/2011,NaN,Social Studies Dept. Meeting,N,Y,Department meeting,Chris Gallagher,cgallagher@schoolwires.com,814-555-5179,High School,2,N,N,25,9/2/2011
1,9/5/2011,6:00:00 PM,9/5/2011,8:00:00 PM,Curriculum Meeting,N,N,Curriculum Meeting,Chris Gallagher,cgallagher@schoolwires.com,814-555-5179,High School,2,N,N,25,9/2/2011


In [113]:
path="sample.csv"
data_csv=tf.contrib.data.make_csv_dataset(path, batch_size=32)
iter = data_csv.make_one_shot_iterator()
next = iter.get_next()
print(next)


OrderedDict([('Start Date ', <tf.Tensor 'IteratorGetNext_34:15' shape=(?,) dtype=string>), ('Start Time', <tf.Tensor 'IteratorGetNext_34:16' shape=(?,) dtype=string>), ('End Date', <tf.Tensor 'IteratorGetNext_34:5' shape=(?,) dtype=string>), ('End Time', <tf.Tensor 'IteratorGetNext_34:6' shape=(?,) dtype=string>), ('Event Title ', <tf.Tensor 'IteratorGetNext_34:8' shape=(?,) dtype=string>), ('All Day Event', <tf.Tensor 'IteratorGetNext_34:0' shape=(?,) dtype=string>), ('No End Time', <tf.Tensor 'IteratorGetNext_34:13' shape=(?,) dtype=string>), ('Event Description', <tf.Tensor 'IteratorGetNext_34:7' shape=(?,) dtype=string>), ('Contact ', <tf.Tensor 'IteratorGetNext_34:2' shape=(?,) dtype=string>), ('Contact Email', <tf.Tensor 'IteratorGetNext_34:3' shape=(?,) dtype=string>), ('Contact Phone', <tf.Tensor 'IteratorGetNext_34:4' shape=(?,) dtype=string>), ('Location', <tf.Tensor 'IteratorGetNext_34:10' shape=(?,) dtype=string>), ('Category', <tf.Tensor 'IteratorGetNext_34:1' shape=(?,) d

In [117]:
inputs, labels = next['End Date'], next['Location']
with  tf.Session() as sess:
    print(sess.run([inputs, labels]))

[array([b'9/5/2011', b'9/5/2011', b'9/5/2011', b'9/5/2011', b'9/5/2011',
       b'9/5/2011', b'9/5/2011', b'9/5/2011', b'9/5/2011', b'9/5/2011',
       b'9/5/2011', b'9/5/2011', b'9/5/2011', b'9/5/2011', b'9/5/2011',
       b'9/5/2011', b'9/5/2011', b'9/5/2011', b'9/5/2011', b'9/5/2011',
       b'9/5/2011', b'9/5/2011', b'9/5/2011', b'9/5/2011', b'9/5/2011',
       b'9/5/2011', b'9/5/2011', b'9/5/2011', b'9/5/2011', b'9/5/2011',
       b'9/5/2011', b'9/5/2011'], dtype=object), array([b'High School', b'High School', b'High School', b'High School',
       b'High School', b'High School', b'High School', b'High School',
       b'High School', b'High School', b'High School', b'High School',
       b'High School', b'High School', b'High School', b'High School',
       b'High School', b'High School', b'High School', b'High School',
       b'High School', b'High School', b'High School', b'High School',
       b'High School', b'High School', b'High School', b'High School',
       b'High School'

In [120]:
#iterator types: one shot,initializable,re-initializable,feedable
data = np.random.sample((100,2))
data[0]

array([0.47194223, 0.00933356])

In [121]:
#Initializable Iterator

x = tf.placeholder(tf.float32, shape=[None,2])
dataset_it = tf.data.Dataset.from_tensor_slices(x)
iter = dataset_it.make_initializable_iterator() # create the iterator
el = iter.get_next()
with tf.Session() as sess:
    # feed the placeholder with data
    sess.run(iter.initializer, feed_dict={ x: data }) 
    print(sess.run(el))

[0.47194225 0.00933356]


In [122]:
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((10,2)), np.random.sample((10,1)))

In [123]:
# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)

In [124]:
# create a iterator of the correct shape and type
iter = tf.data.Iterator.from_structure(train_dataset.output_types,train_dataset.output_shapes)

In [125]:
# create the initialisation operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)

In [126]:
features, labels = iter.get_next()

In [127]:
# Reinitializable iterator to switch between Datasets
EPOCHS = 10
# making fake data using numpy
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((10,2)), np.random.sample((10,1)))
# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)
# create a iterator of the correct shape and type
iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
features, labels = iter.get_next()
# create the initialisation operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)
with tf.Session() as sess:
    sess.run(train_init_op) # switch to train dataset
    for _ in range(EPOCHS):
        sess.run([features, labels])
    sess.run(test_init_op) # switch to val dataset
    print(sess.run([features, labels]))

[array([0.90332682, 0.0697792 ]), array([0.54479065])]
